# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Для выполнения задания необходмимо спрогнозировать уйдет ли клиент Бета-Банка в ближайшее время на основе исторических данных о поведении клиентов и расторжении договоров. Нужно построить модель с большим значением F1-меры, не менее 0.59. Дополнительно измерить AUC-ROC.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

Проанализируем исходные данные:

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В столбце Tenure есть пропуски, заменим их медианным значением

In [4]:
data.loc[data['Tenure'].isna()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [5]:
data.loc[data['Tenure'].isna(), 'Tenure'] = data['Tenure'].median()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


## Исследование задачи

Выясним, есть ли дисбаланс классов в наборе исследуемых данных

In [7]:
data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Дисбаланс 80 на 20

Преобразуем категориальные признаки в численные: это столбцы Gender и Geography. Удалим RowNumber, CustomerId, Surname для исследования, они никак не должны влиять на модель

In [8]:
data_ohe = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data_ohe = pd.get_dummies(data_ohe, drop_first=True)
data_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


Обучим модель логистической регрессии, но перед этим разделим датасет на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [9]:
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

print('Тренировочная выборка:', len(target_train))
print('Валидационная выборка:', len(target_valid))
print('Тестовая выборка:', len(target_test))

Тренировочная выборка: 6000
Валидационная выборка: 2000
Тестовая выборка: 2000


Применим масштабирование для столбцов с численными значениями:

In [10]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

features_train_scaled = features_train
features_valid_scaled = features_valid
features_test_scaled = features_test
print(features_train_scaled.shape)

scaler = StandardScaler()
scaler.fit(features_train_scaled[numeric])

features_train_scaled[numeric] = scaler.transform(features_train[numeric])
features_valid_scaled[numeric] = scaler.transform(features_valid[numeric])
features_test_scaled[numeric] = scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None

print(features_train_scaled.shape)

(6000, 11)
(6000, 11)


/tmp/ipykernel_477/358330891.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train_scaled[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [11]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.33108108108108103


При обучении модели на "сырых" исходных данных только с учетом масштабирования получено низкое значение F1 = 0.33 по причине дисбаланса данных.

## Борьба с дисбалансом

Для начала попробуем сбалансировать датасет при помощи class_weight='balanced'

In [12]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid_scaled)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC = ', auc_roc)

F1: 0.4888507718696398
AUC-ROC =  0.7637310291013133


При таком подходе F1 повысилось с 0.33 до 0.49, но этого не достаточно.

Попробуем увеличить выборку в х раз:

In [13]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = np.concatenate([features_zeros] + [features_ones] * repeat)
    target_upsampled = np.concatenate([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

best_f1 = 0
current_f1 = 0
for n in range(1, 10):
    features_upsampled, target_upsampled = upsample(features_train_scaled, target_train, n)

    model = LogisticRegression(random_state=12345, solver='liblinear')
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    current_f1 = f1_score(target_valid, predicted_valid)
    if best_f1 < current_f1:
        best_f1 = current_f1
        best_repeat = n

print("Лучший F1:", best_f1)
print("Лучшее увеличение выборки:", best_repeat)

features_train_upsampled, target_train_upsampled = upsample(features_train_scaled, target_train, best_repeat)
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train_upsampled, target_train_upsampled)

probabilities_valid = model.predict_proba(features_valid_scaled)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC = ', auc_roc)

Лучший F1: 0.5
Лучшее увеличение выборки: 3
AUC-ROC =  0.7626815429563449


При увеличении выборки F1 увеличилось незначительно по сравнению с class_weight='balanced' и равно 0.5

Для полученных наборов данных попробуем применить другие модели, например модель леса.
Подберем оптимальные параметры:

In [14]:
best_f1 = 0
current_f1 = 0

for depth in range(1, 10):
    model = RandomForestClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_upsampled, target_train_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    current_f1 = f1_score(target_valid, predicted_valid)
    if best_f1 < current_f1:
        best_f1 = current_f1
        best_depth = depth

print("Лучший F1:", f1_score(target_valid, predicted_valid))
print("Лучшая глубина леса:", best_depth)


model = RandomForestClassifier(random_state=12345, max_depth=best_depth)
model.fit(features_train_upsampled, target_train_upsampled)

probabilities_valid = model.predict_proba(features_valid_scaled)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC = ', auc_roc)

Лучший F1: 0.6273148148148147
Лучшая глубина леса: 6
AUC-ROC =  0.8524179011486884


In [15]:
best_f1 = 0
current_f1 = 0

for n in range(10, 110, 10):
    model = RandomForestClassifier(random_state=12345, max_depth=best_depth, n_estimators=n)
    model.fit(features_train_upsampled, target_train_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    current_f1 = f1_score(target_valid, predicted_valid)
    if best_f1 < current_f1:
        best_f1 = current_f1
        best_n_estimators = n

print("Лучший F1:", f1_score(target_valid, predicted_valid))
print("Лучшее кол-во деревьев леса:", best_n_estimators)


model = RandomForestClassifier(random_state=12345, max_depth=best_depth, n_estimators=best_n_estimators)
model.fit(features_train_upsampled, target_train_upsampled)

probabilities_valid = model.predict_proba(features_valid_scaled)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('AUC-ROC = ', auc_roc)

Лучший F1: 0.6381502890173409
Лучшее кол-во деревьев леса: 100
AUC-ROC =  0.8524179011486884


Наилучшие показатели F1 = 0.63 и AUC-ROC = 0.85 достигнуты моделью леса с максимальной глубиной 6 и количеством деревьев = 100.
Протестируем это модель на тестовой выборке.

## Тестирование модели

In [16]:
predicted_test = model.predict(features_test_scaled)
test_f1 = f1_score(target_test, predicted_test)
print("F1 тестовой модели:", test_f1)

probabilities_test = model.predict_proba(features_test_scaled)
probabilities_one_valid = probabilities_test[:, 1]

auc_roc = roc_auc_score(target_test, probabilities_one_valid)

print('AUC-ROC = ', auc_roc)

F1 тестовой модели: 0.5972222222222222
AUC-ROC =  0.8516199924745641


Для "Бета-Банка" построена модель Случайного леса, предсказывающая уход клиентов с F1=0.6 и AUC-ROC = 0.85.
При подготовке данных применено масштабирование столбцов с численными значениями, выбран оптимальный способ избавления от дисбаланса данных: увеличение тренировочной выборки в 4 раза. Далее применена модель Случайного леса с максимальной глубиной 6 и количеством деревьев = 100